In [28]:
from tensorflow import keras
from tensorflow.keras import layers
import kerastuner
from kerastuner.tuners import RandomSearch


### Test on IRIS dataset

In [3]:
from sklearn.datasets import load_iris
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

iris_ds = load_iris()
X = iris_ds.data
X[:5]


array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [6]:
# convert integers to dummy variables (i.e. one hot encoded)
y_one_hot_encoded = to_categorical(iris_ds.target)
y_one_hot_encoded[:3]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot_encoded, test_size=0.33, random_state=42)


Usage: the basics
Here's how to perform hyperparameter tuning for a single-layer dense neural network using random search.

First, we define a model-building function. It takes an argument hp from which you can sample hyperparameters, such as hp.Int('units', min_value=32, max_value=512, step=32) (an integer from a certain range).

This function returns a compiled model.

In [8]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(input_dim=X.shape[1],
                            units=hp.Int('units',
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                            
                           activation='relu'))
    model.add(layers.Dense(3, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

Next, instantiate a tuner. You should specify the model-building function, the name of the objective to optimize (whether to minimize or maximize is automatically inferred for built-in metrics), the total number of trials (max_trials) to test, and the number of models that should be built and fit for each trial (executions_per_trial).

Available tuners are RandomSearch and Hyperband.

Note: the purpose of having multiple executions per trial is to reduce results variance and therefore be able to more accurately assess the performance of a model. If you want to get results faster, you could set executions_per_trial=1 (single round of training for each model configuration).

In [12]:
import os

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory=os.path.normpath('/Users/germancabrera/Documents/GitRepos/DeepLearning_concepts_checks/project/KERAS_TUNER_CHECKS'),
    project_name='iris_data_keras_tuner_test_mac')


summary of the search space:

In [13]:
tuner.search_space_summary()


Start the search for the best hyperparameter configuration. The call to search has the same signature as model.fit().
Here's what happens in search: models are built iteratively by calling the model-building function, which populates the hyperparameter space (search space) tracked by the hp object. The tuner progressively explores the space, recording metrics for each configuration.

In [18]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test)
             #validation_split=0.2,verbose=1)
             )


INFO:tensorflow:Oracle triggered exit


In [8]:
tuner.results_summary()


## Best models achieved with the hyperparametrization

In [15]:
models = tuner.get_best_models(num_models=2)


## Evaluation score:

In [16]:
print('single prediction on a test instance: {}'.format(models[0].predict(X_test[-1].reshape(1, -1)))) 


single prediction on a test instance: [[1.1654331e-04 3.6200428e-01 6.3787913e-01]]


## Predictions on the test set:

In [17]:
test_set_predictions = models[0].predict(X_test)


### Esperamos obtener un score = 0.90 para el best_model_0 según el 'Results summary'

In [19]:
import tensorflow

categ_acc_test_set = tensorflow.keras.metrics.CategoricalAccuracy()

#best_model_0_eval_set_acc = tensorflow.keras.metrics.CategoricalAccuracy(y_test.reshape(-1,3), test_set_predictions.reshape(-1,3))
_ = categ_acc_test_set.update_state(y_test, test_set_predictions)
print('best_model_0_eval_set_acc: {}'.format(categ_acc_test_set.result().numpy()))


best_model_0_eval_set_acc: 0.9800000190734863


## Y con precision-recall metrics?

In [42]:
from tensorflow.keras import metrics

def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(input_dim=X.shape[1],
                            units=hp.Int('units',
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                            
                           activation='relu'))
    model.add(layers.Dense(3, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=[metrics.Recall(thresholds=0.8)]) #, metrics.Precision()])
    return model

In [43]:
import os

tuner = RandomSearch(
    build_model,
    objective=kerastuner.Objective("val_recall", direction="max"), #'val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory=os.path.normpath('/Users/germancabrera/Documents/GitRepos/DeepLearning_concepts_checks/project/KERAS_TUNER_CHECKS'),
    project_name='iris_data_keras_tuner_test_mac')


INFO:tensorflow:Reloading Oracle from existing project /Users/germancabrera/Documents/GitRepos/DeepLearning_concepts_checks/project/KERAS_TUNER_CHECKS/iris_data_keras_tuner_test_mac/oracle.json
INFO:tensorflow:Reloading Tuner from /Users/germancabrera/Documents/GitRepos/DeepLearning_concepts_checks/project/KERAS_TUNER_CHECKS/iris_data_keras_tuner_test_mac/tuner0.json


In [44]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test)
             #validation_split=0.2,verbose=1)
             )


INFO:tensorflow:Oracle triggered exit


In [31]:
tuner.results_summary()

In [45]:
tuner.results_summary()

In [33]:
m = metrics.Recall()
m.reset_states()
m.update_state([0, 1, 1, 1], [1, 0, 1, 1]) #, sample_weight=[0, 0, 1, 0])
m.result().numpy()

0.6666667

In [34]:
recall_a_mano = 2/3
recall_a_mano

0.6666666666666666

In [35]:
# ahora espero obtener el mismo valor usando el default threshold de 0.5
m = metrics.Recall()
m.reset_states()
m.update_state([0, 1, 1, 1], [1, 0.3, 0.8, 0.9]) #, sample_weight=[0, 0, 1, 0])
m.result().numpy()

0.6666667

In [37]:
# ahora con threshold de 0.8 obtendré menor recall con las mismas predicciones
m = metrics.Recall(thresholds=0.85)
m.reset_states()
m.update_state([0, 1, 1, 1], [1, 0.3, 0.8, 0.9]) #, sample_weight=[0, 0, 1, 0])
m.result().numpy()

0.33333334

cuadra al haber acertado un único true positive

### Y con el train set?

In [13]:
categ_acc_train_set = tensorflow.keras.metrics.CategoricalAccuracy()

train_set_predictions = models[0].predict(X_train)
_ = categ_acc_train_set.update_state(y_train, train_set_predictions)
print('best_model_0_train_set_acc: {}'.format(categ_acc_train_set.result().numpy()))


best_model_0_train_set_acc: 0.949999988079071


### Obtengo resultado de accuracy mejor que el 0.90 esperado, pruebo si al menos el mejor modelo es el [0]

In [14]:
import tensorflow

models = tuner.get_best_models(num_models=5)

test_set_predictions_0 = models[0].predict(X_test)
test_set_predictions_1 = models[1].predict(X_test)
test_set_predictions_2 = models[2].predict(X_test)
test_set_predictions_3 = models[3].predict(X_test)
test_set_predictions_4 = models[4].predict(X_test)


In [15]:
categ_acc_test_set = tensorflow.keras.metrics.CategoricalAccuracy()

#best_model_0_eval_set_acc = tensorflow.keras.metrics.CategoricalAccuracy(y_test.reshape(-1,3), test_set_predictions.reshape(-1,3))
_ = categ_acc_test_set.update_state(y_test, test_set_predictions_0)
print('best_model_0_eval_set_acc: {}'.format(categ_acc_test_set.result().numpy()))

_ = categ_acc_test_set.update_state(y_test, test_set_predictions_1)
print('best_model_0_eval_set_acc: {}'.format(categ_acc_test_set.result().numpy()))

_ = categ_acc_test_set.update_state(y_test, test_set_predictions_2)
print('best_model_0_eval_set_acc: {}'.format(categ_acc_test_set.result().numpy()))

_ = categ_acc_test_set.update_state(y_test, test_set_predictions_3)
print('best_model_0_eval_set_acc: {}'.format(categ_acc_test_set.result().numpy()))

_ = categ_acc_test_set.update_state(y_test, test_set_predictions_4)
print('best_model_0_eval_set_acc: {}'.format(categ_acc_test_set.result().numpy()))



best_model_0_eval_set_acc: 1.0
best_model_0_eval_set_acc: 0.8600000143051147
best_model_0_eval_set_acc: 0.800000011920929
best_model_0_eval_set_acc: 0.7699999809265137
best_model_0_eval_set_acc: 0.6759999990463257


In [16]:
models = tuner.get_best_models(num_models=1)
test_set_predictions_0 = models[0].predict(X_test)
categ_acc_test_set = tensorflow.keras.metrics.CategoricalAccuracy()
_ = categ_acc_test_set.update_state(y_test, test_set_predictions_0)
print('best_model_0_eval_set_acc: {}'.format(categ_acc_test_set.result().numpy()))



best_model_0_eval_set_acc: 1.0


## Hasta aquí hemos podido utilizar un RANDOM_SEARCH hiperparametrizador keras tuner
### ToDo: acceder a los pesos y arquitectura del modelo